In [2]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Input, Layer
import tensorflow.keras.backend as K

2024-12-20 13:31:00.706138: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 13:31:00.823439: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 13:31:00.911654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734681660.999035   33929 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734681661.027125   33929 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 13:31:01.237165: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [4]:
X_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_train.csv", header=None)
X_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_test.csv", header=None)
y_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_train.csv", header=None)
y_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_test.csv", header=None)



print("X train ",X_train.shape)
print("X test",X_test.shape)
print("Y_Train", y_train.shape)
print("Y_test", y_test.shape)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

X train  (3206, 2000)
X test (375, 2000)
Y_Train (3206, 1)
Y_test (375, 1)


In [7]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
print("X train ",X_train.shape)
print("X test",X_test.shape)
print("Y_Train", y_train.shape)
print("Y_test", y_test.shape)

X train  (3206, 2000, 1)
X test (375, 2000, 1)
Y_Train (3206, 1)
Y_test (375, 1)


In [3]:


# Custom Attention Layer
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(
            name="attention_weight",
            shape=(input_shape[-1], 1),
            initializer="random_normal",
            trainable=True,
        )
        super(Attention, self).build(input_shape)

    def call(self, inputs):
        score = K.tanh(K.dot(inputs, self.W))
        attention_weights = K.softmax(score, axis=1)
        context_vector = K.sum(inputs * attention_weights, axis=1)
        return context_vector

model = Sequential()

model.add(Input(shape=(1281, 1))) # 1281 timestamp with 1 feature each.

model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer="l2")))
model.add(Dropout(0.3))

model.add(Attention())

model.add(Dense(64, activation="relu", kernel_regularizer="l2"))
model.add(Dropout(0.3))
model.add(Dense(32, activation="relu", kernel_regularizer="l2"))
model.add(Dropout(0.3))

model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()


2024-12-20 13:31:07.868578: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 1281, 256)      │       133,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1281, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention (Attention)           │ (None, 256)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,937 (593.50 KB)

 Trainable params: 151,937 (593.50 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
import keras
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
)

def compile_model(model):

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

def fit_model(model, epochs, X_train, y_train,
              X_test, y_test, ):

    history = model.fit(X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=epochs,
                batch_size=64,
                verbose=1)

    return history


def get_classification_metrics(y_test: pd.DataFrame, y_pred: pd.DataFrame) -> dict[str, str]:
    """
    Returns the classification report as a dictionary.
    """
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred),
        "fb_score": fbeta_score(y_test, y_pred, beta=0.2),
    }


In [6]:
history = fit_model(model, 5, X_train, y_train, X_test, y_test)


Epoch 1/5


2024-12-02 10:19:41.050448: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 83951616 exceeds 10% of free system memory.
2024-12-02 10:19:41.083256: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 83951616 exceeds 10% of free system memory.
2024-12-02 10:19:41.146049: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 83951616 exceeds 10% of free system memory.
2024-12-02 10:19:41.174121: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 83951616 exceeds 10% of free system memory.
2024-12-02 10:19:41.183466: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 83951616 exceeds 10% of free system memory.


21/21 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.5778 - loss: 2.0197 - val_accuracy: 0.1013 - val_loss: 1.9194
Epoch 2/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5565 - loss: 1.6274 - val_accuracy: 0.1040 - val_loss: 1.4358
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.5444 - loss: 1.3736 - val_accuracy: 0.1040 - val_loss: 1.2396
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5898 - loss: 1.1878 - val_accuracy: 0.1067 - val_loss: 1.3741
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.8280 - loss: 0.9558 - val_accuracy: 0.1013 - val_loss: 3.6384


In [7]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)
get_classification_metrics(y_test, y_pred_binary)

12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 399ms/step


{'accuracy': 0.10133333333333333,
 'precision': np.float64(0.10133333333333333),
 'recall': np.float64(1.0),
 'f1_score': np.float64(0.18401937046004843),
 'confusion_matrix': array([[  0, 337],
        [  0,  38]]),
 'fb_score': np.float64(0.10496122383937109)}

In [9]:
import dagshub
import mlflow
dagshub.init(repo_owner='pavipd495', repo_name='spam_or_ham_fsec', mlflow=True)
mlflow.set_experiment("Spam_Or_Ham")


with mlflow.start_run(run_name="dl_lstm_att"):

    # # Get the predictions.
    # y_pred = model.predict(X_test)
    # y_pred = (y_pred >= 0.5).astype(int)

    # Log metrics to mlflow.
    metrics = get_classification_metrics(y_test=y_test, y_pred=y_pred_binary)
    mlflow.log_metric("accuracy", metrics["accuracy"],)
    mlflow.log_metric("precision", metrics["precision"])
    mlflow.log_metric("recall", metrics["recall"])
    mlflow.log_metric("f1_score", metrics["f1_score"])

    mlflow.keras.log_model(model, "lstm_att")

Initialized MLflow to track repo "pavipd495/spam_or_ham_fsec"

Repository pavipd495/spam_or_ham_fsec initialized!

2024/12/02 10:24:27 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/12/02 10:24:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_m3y0j9w/model, flavor: keras). Fall back to return ['keras==3.7.0']. Set logging level to DEBUG to see the full traceback. 
2024/12/02 10:24:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run dl_lstm_att at: https://dagshub.com/pavipd495/spam_or_ham_fsec.mlflow/#/experiments/2/runs/723a826c90dc4f1e95c068a85c2d1f92
🧪 View experiment at: https://dagshub.com/pavipd495/spam_or_ham_fsec.mlflow/#/experiments/2
